In [1]:
# http request
import requests
from bs4 import BeautifulSoup
import numpy as np
import pymysql
import pandas as pd
from datetime import datetime

In [5]:
# http get request
req = requests.get('https://www.10000recipe.com/recipe/6854799')

In [6]:
# html 소스 가져오기
html = req.text
# BeautifulSoup로 html 소스를 python객체로 변환하기
# 첫 인자는 html소스코드, 두 번째 인자는 어떤 parser를 이용할지 명시,
# 이 글에서는 python 내장 html.parser를 이용했다.
soup = BeautifulSoup(html, 'html.parser')

In [7]:
# CSS Selector를 통해 html요소들을 찾아낸다.
#contents_area > div.view2_summary > h3
# 타이틀 제목
def titles():
    my_titles = soup.select(
    '#contents_area > div.view2_summary > h3'
    )[0].text
    return my_titles
titles()

'닭가슴살 갈비구이 '

In [8]:
# 음식양, 조리 시간, 조리 난이도 가져오기.
#contents_area > div.view2_summary > div.view2_summary_info > span.view2_summary_info(1,2,3)
# 정보
def information():
    info_box = []
    for n in range(1,4):
        info = soup.select(
        '#contents_area > div.view2_summary > div.view2_summary_info > span.view2_summary_info%i' %n
        )[0].text
        if n == 1:
            info1 = info
        elif n == 2:
            info2 = info
        else:
            info3 = info
#         info_box.append(info)
    return info1, info2, info3
information()

('1인분', '30분 이내', '아무나')

In [9]:
# 음식 재료 및 재료 양 가져오기
#divConfirmedMaterialArea > ul:nth-child(1)
#divConfirmedMaterialArea > ul:nth-child(1) > li:nth-child(2)
#divConfirmedMaterialArea > ul:nth-child(1) > li:nth-child(3)
#divConfirmedMaterialArea > ul:nth-child(2)
#divConfirmedMaterialArea > ul:nth-child(2) > li:nth-child(2)
#divConfirmedMaterialArea > ul:nth-child(2) > li:nth-child(3)
def meterial_meterial(num):
    me_box = []
    me_sea = []
    for m in range(1,num+1):
        me1 = soup.select(
        '#divConfirmedMaterialArea > ul:nth-child(%s)'%m
        )
        a = str(me1).count('<li>')
        a_a = a+1
        for b in range(2,a_a+1):
            me2 = soup.select(
            '#divConfirmedMaterialArea > ul:nth-child(%s) > li:nth-child(%s)'%(m,b)
            )[0]
            me2_text = me2.text
            me2_text = me2_text.replace(" ","")
            me2_text = me2_text.replace("\n"," ")
            me2_text = me2_text.strip()
            if m == 1:
                me_box.append(me2_text)
            else:
                me_sea.append(me2_text)
    return me_box, me_sea
meterial_meterial(2)

(['닭가슴살 200g', '대파 1개'], ['간장 2T', '설탕 1T', '참기름 0.5T', '다진마늘 0.5T', '꿀 1T'])

In [10]:
# 요리 후기

#contents_area > div.view_reply.st2 > div.reply_tit > span
def score_count():
    try:
        global score_num
        score_n = soup.select(
        '#contents_area > div.view_reply.st2 > div.reply_tit > span'
        )[0].text
        score_num = float(score_n)
        return score_num
    except:
        score_n = soup.select(
        '#recipeCommentListCount'
        )[0].text
        score_num = float(score_n)
        return score_num
score_count()

102.0

In [11]:
# 요리 별점
#contents_area > div.view_reply.st2 > div:nth-child(2) > div:nth-child(1) > div.media-body > h4 > span
#contents_area > div.view_reply.st2 > div:nth-child(2) > div:nth-child(2) > div.media-body > h4 > span
def score_mean():
    try:
        score = soup.select(
        '#contents_area > div.view_reply.st2 > div:nth-child(2)'
        )
        score_str = str(score)
        score_c = score_str.count('https://recipe1.ezmember.co.kr/img/mobile/icon_star2_on.png')
        score_m = round(score_c/score_num,2)
        score_m
        return score_m
    except:
        return 0
score_mean()

4.87

In [12]:
# 조회수 가져오기
#contents_area > div.view2_pic > div.view_cate.st2 > div > span
def view_num():
    views = soup.select(
    '#contents_area > div.view2_pic > div.view_cate.st2 > div > span'
    )[0].text
    return views
view_num()

'122,680'

In [13]:
# 조리 순서 count
def cook_num():
    cook_seq = soup.select(
    '#contents_area > div:nth-child(11)'
    )[0]
    cook_str = str(cook_seq)
    cook_n = cook_str.count('view_step_cont media step')
    cook_n
    return cook_n
cook_num()

8

# 전체 연결

In [17]:
# html 소스 가져오기
html = req.text
# BeautifulSoup로 html 소스를 python객체로 변환하기
# 첫 인자는 html소스코드, 두 번째 인자는 어떤 parser를 이용할지 명시,
# 이 글에서는 python 내장 html.parser를 이용했다.
# count: 98713
# error : 5343
# result: 93370

soup = BeautifulSoup(html, 'html.parser')

col_name = ['제목', '음식양', '조리시간', '조리난이도', '재료',
            '양념', '요리후기', '별점평균', '조회수' ,'조리순서']

data_f = pd.DataFrame(columns = col_name)

error_num = 0
count_n = 0
count = 100
per_n = 6927521 - 6828809
for num in range(6828809, 6927521+1):
    now = datetime.now()
    base_url = 'https://www.10000recipe.com/recipe/{}'
    url = base_url.format(num)
    req = requests.get(url)
    html = req.text
    soup = BeautifulSoup(html, 'html.parser')
    
    try:
        titles()
        information()
        meterial_meterial(2)
        score_count()
        score_mean()
        view_num()
        cook_num()
        
        data = [[titles(), information()[0], information()[1], information()[2], meterial_meterial(2)[0],
        meterial_meterial(2)[1], score_count(), score_mean(), view_num(), cook_num()]]
        
        df = pd.DataFrame(data = data, columns = col_name)
        data_f = data_f.append(df)
    except:
        error_num += 1
    count_n += 1
    progress = count_n % count
    if progress == 0:
        print('progress... %s\t error... %s\t Percent... %0.2f%% \t%s'%(count_n, error_num, count_n/per_n*100, str(now)))
print('''
Count: %s
Error: %s
Result: %s'''%(count_n, error_num, count_n-error_num))

progress... 100	 error... 11	 Percent... 0.10% 	2020-03-06 21:07:21.279993
progress... 200	 error... 46	 Percent... 0.20% 	2020-03-06 21:08:14.754076
progress... 300	 error... 52	 Percent... 0.30% 	2020-03-06 21:09:18.652339
progress... 400	 error... 62	 Percent... 0.41% 	2020-03-06 21:10:22.217836
progress... 500	 error... 68	 Percent... 0.51% 	2020-03-06 21:11:24.689892
progress... 600	 error... 78	 Percent... 0.61% 	2020-03-06 21:12:26.126631
progress... 700	 error... 86	 Percent... 0.71% 	2020-03-06 21:13:29.690776
progress... 800	 error... 98	 Percent... 0.81% 	2020-03-06 21:14:30.781499
progress... 900	 error... 110	 Percent... 0.91% 	2020-03-06 21:15:30.592597
progress... 1000	 error... 115	 Percent... 1.01% 	2020-03-06 21:16:32.134085
progress... 1100	 error... 124	 Percent... 1.11% 	2020-03-06 21:17:35.522132
progress... 1200	 error... 128	 Percent... 1.22% 	2020-03-06 21:18:42.490115
progress... 1300	 error... 131	 Percent... 1.32% 	2020-03-06 21:19:50.500467
progress... 1400

progress... 10800	 error... 477	 Percent... 10.94% 	2020-03-06 23:13:11.561021
progress... 10900	 error... 479	 Percent... 11.04% 	2020-03-06 23:14:23.124989
progress... 11000	 error... 481	 Percent... 11.14% 	2020-03-06 23:15:32.724349
progress... 11100	 error... 484	 Percent... 11.24% 	2020-03-06 23:16:42.218645
progress... 11200	 error... 488	 Percent... 11.35% 	2020-03-06 23:17:50.437698
progress... 11300	 error... 489	 Percent... 11.45% 	2020-03-06 23:19:01.044259
progress... 11400	 error... 492	 Percent... 11.55% 	2020-03-06 23:20:12.274864
progress... 11500	 error... 494	 Percent... 11.65% 	2020-03-06 23:21:21.246038
progress... 11600	 error... 496	 Percent... 11.75% 	2020-03-06 23:22:30.270503
progress... 11700	 error... 498	 Percent... 11.85% 	2020-03-06 23:23:41.637119
progress... 11800	 error... 502	 Percent... 11.95% 	2020-03-06 23:24:50.004793
progress... 11900	 error... 505	 Percent... 12.06% 	2020-03-06 23:25:58.294246
progress... 12000	 error... 515	 Percent... 12.16% 	

progress... 21200	 error... 1115	 Percent... 21.48% 	2020-03-07 01:14:05.085522
progress... 21300	 error... 1121	 Percent... 21.58% 	2020-03-07 01:15:14.730732
progress... 21400	 error... 1129	 Percent... 21.68% 	2020-03-07 01:16:22.553427
progress... 21500	 error... 1134	 Percent... 21.78% 	2020-03-07 01:17:30.986519
progress... 21600	 error... 1140	 Percent... 21.88% 	2020-03-07 01:18:41.493048
progress... 21700	 error... 1151	 Percent... 21.98% 	2020-03-07 01:19:44.268207
progress... 21800	 error... 1159	 Percent... 22.08% 	2020-03-07 01:20:51.278078
progress... 21900	 error... 1162	 Percent... 22.19% 	2020-03-07 01:21:58.980099
progress... 22000	 error... 1167	 Percent... 22.29% 	2020-03-07 01:23:06.900534
progress... 22100	 error... 1175	 Percent... 22.39% 	2020-03-07 01:24:12.794418
progress... 22200	 error... 1180	 Percent... 22.49% 	2020-03-07 01:25:20.226130
progress... 22300	 error... 1182	 Percent... 22.59% 	2020-03-07 01:26:31.295150
progress... 22400	 error... 1189	 Percen

progress... 31500	 error... 2158	 Percent... 31.91% 	2020-03-07 03:11:40.618839
progress... 31600	 error... 2161	 Percent... 32.01% 	2020-03-07 03:12:46.935562
progress... 31700	 error... 2165	 Percent... 32.11% 	2020-03-07 03:13:58.384566
progress... 31800	 error... 2171	 Percent... 32.21% 	2020-03-07 03:15:01.464940
progress... 31900	 error... 2179	 Percent... 32.32% 	2020-03-07 03:16:06.294667
progress... 32000	 error... 2182	 Percent... 32.42% 	2020-03-07 03:17:15.835773
progress... 32100	 error... 2189	 Percent... 32.52% 	2020-03-07 03:18:18.666784
progress... 32200	 error... 2192	 Percent... 32.62% 	2020-03-07 03:19:21.252481
progress... 32300	 error... 2201	 Percent... 32.72% 	2020-03-07 03:20:23.125084
progress... 32400	 error... 2204	 Percent... 32.82% 	2020-03-07 03:21:25.300876
progress... 32500	 error... 2213	 Percent... 32.92% 	2020-03-07 03:22:29.452387
progress... 32600	 error... 2228	 Percent... 33.03% 	2020-03-07 03:23:37.426713
progress... 32700	 error... 2233	 Percen

progress... 41800	 error... 2714	 Percent... 42.35% 	2020-03-07 05:14:48.955453
progress... 41900	 error... 2722	 Percent... 42.45% 	2020-03-07 05:15:59.799074
progress... 42000	 error... 2725	 Percent... 42.55% 	2020-03-07 05:17:12.722168
progress... 42100	 error... 2726	 Percent... 42.65% 	2020-03-07 05:18:29.404152
progress... 42200	 error... 2727	 Percent... 42.75% 	2020-03-07 05:19:40.616788
progress... 42300	 error... 2729	 Percent... 42.85% 	2020-03-07 05:20:58.681108
progress... 42400	 error... 2732	 Percent... 42.95% 	2020-03-07 05:22:10.630801
progress... 42500	 error... 2736	 Percent... 43.05% 	2020-03-07 05:23:22.934492
progress... 42600	 error... 2737	 Percent... 43.16% 	2020-03-07 05:24:43.182007
progress... 42700	 error... 2742	 Percent... 43.26% 	2020-03-07 05:25:56.037220
progress... 42800	 error... 2745	 Percent... 43.36% 	2020-03-07 05:27:08.692997
progress... 42900	 error... 2746	 Percent... 43.46% 	2020-03-07 05:28:21.857444
progress... 43000	 error... 2751	 Percen

progress... 52100	 error... 3184	 Percent... 52.78% 	2020-03-07 07:19:34.610947
progress... 52200	 error... 3189	 Percent... 52.88% 	2020-03-07 07:20:48.357775
progress... 52300	 error... 3194	 Percent... 52.98% 	2020-03-07 07:21:59.659205
progress... 52400	 error... 3197	 Percent... 53.08% 	2020-03-07 07:23:14.206895
progress... 52500	 error... 3199	 Percent... 53.19% 	2020-03-07 07:24:27.012303
progress... 52600	 error... 3205	 Percent... 53.29% 	2020-03-07 07:25:39.598236
progress... 52700	 error... 3207	 Percent... 53.39% 	2020-03-07 07:26:56.021942
progress... 52800	 error... 3211	 Percent... 53.49% 	2020-03-07 07:28:09.921394
progress... 52900	 error... 3213	 Percent... 53.59% 	2020-03-07 07:29:20.785989
progress... 53000	 error... 3219	 Percent... 53.69% 	2020-03-07 07:30:30.703090
progress... 53100	 error... 3226	 Percent... 53.79% 	2020-03-07 07:31:45.983820
progress... 53200	 error... 3233	 Percent... 53.89% 	2020-03-07 07:32:55.903910
progress... 53300	 error... 3239	 Percen

progress... 62400	 error... 3752	 Percent... 63.21% 	2020-03-07 09:26:38.904805
progress... 62500	 error... 3757	 Percent... 63.32% 	2020-03-07 09:27:53.759698
progress... 62600	 error... 3760	 Percent... 63.42% 	2020-03-07 09:29:06.753574
progress... 62700	 error... 3767	 Percent... 63.52% 	2020-03-07 09:30:20.070583
progress... 62800	 error... 3772	 Percent... 63.62% 	2020-03-07 09:31:40.475616
progress... 62900	 error... 3783	 Percent... 63.72% 	2020-03-07 09:32:50.870469
progress... 63000	 error... 3795	 Percent... 63.82% 	2020-03-07 09:34:00.138271
progress... 63100	 error... 3801	 Percent... 63.92% 	2020-03-07 09:35:11.945347
progress... 63200	 error... 3810	 Percent... 64.02% 	2020-03-07 09:36:27.710783
progress... 63300	 error... 3824	 Percent... 64.13% 	2020-03-07 09:37:34.677768
progress... 63400	 error... 3831	 Percent... 64.23% 	2020-03-07 09:38:49.114812
progress... 63500	 error... 3838	 Percent... 64.33% 	2020-03-07 09:40:02.665202
progress... 63600	 error... 3844	 Percen

progress... 72700	 error... 4377	 Percent... 73.65% 	2020-03-07 11:33:50.516096
progress... 72800	 error... 4379	 Percent... 73.75% 	2020-03-07 11:35:07.932118
progress... 72900	 error... 4382	 Percent... 73.85% 	2020-03-07 11:36:26.947922
progress... 73000	 error... 4386	 Percent... 73.95% 	2020-03-07 11:37:41.284208
progress... 73100	 error... 4390	 Percent... 74.05% 	2020-03-07 11:38:58.322242
progress... 73200	 error... 4394	 Percent... 74.16% 	2020-03-07 11:40:14.953393
progress... 73300	 error... 4396	 Percent... 74.26% 	2020-03-07 11:41:28.908729
progress... 73400	 error... 4400	 Percent... 74.36% 	2020-03-07 11:42:47.518588
progress... 73500	 error... 4402	 Percent... 74.46% 	2020-03-07 11:44:06.053650
progress... 73600	 error... 4405	 Percent... 74.56% 	2020-03-07 11:45:19.325750
progress... 73700	 error... 4409	 Percent... 74.66% 	2020-03-07 11:46:37.609513
progress... 73800	 error... 4412	 Percent... 74.76% 	2020-03-07 11:47:49.233502
progress... 73900	 error... 4415	 Percen

progress... 83000	 error... 4889	 Percent... 84.08% 	2020-03-07 13:40:15.551082
progress... 83100	 error... 4892	 Percent... 84.18% 	2020-03-07 13:41:22.937271
progress... 83200	 error... 4892	 Percent... 84.29% 	2020-03-07 13:42:42.810726
progress... 83300	 error... 4895	 Percent... 84.39% 	2020-03-07 13:44:00.749424
progress... 83400	 error... 4900	 Percent... 84.49% 	2020-03-07 13:45:13.779434
progress... 83500	 error... 4902	 Percent... 84.59% 	2020-03-07 13:46:29.560856
progress... 83600	 error... 4906	 Percent... 84.69% 	2020-03-07 13:47:44.103782
progress... 83700	 error... 4910	 Percent... 84.79% 	2020-03-07 13:48:59.028523
progress... 83800	 error... 4916	 Percent... 84.89% 	2020-03-07 13:50:14.905113
progress... 83900	 error... 4919	 Percent... 84.99% 	2020-03-07 13:51:29.592675
progress... 84000	 error... 4922	 Percent... 85.10% 	2020-03-07 13:52:43.209882
progress... 84100	 error... 4924	 Percent... 85.20% 	2020-03-07 13:54:01.102814
progress... 84200	 error... 4928	 Percen

progress... 93300	 error... 5200	 Percent... 94.52% 	2020-03-07 15:48:30.141867
progress... 93400	 error... 5201	 Percent... 94.62% 	2020-03-07 15:49:44.265729
progress... 93500	 error... 5204	 Percent... 94.72% 	2020-03-07 15:51:03.786406
progress... 93600	 error... 5205	 Percent... 94.82% 	2020-03-07 15:52:25.424203
progress... 93700	 error... 5205	 Percent... 94.92% 	2020-03-07 15:53:45.247926
progress... 93800	 error... 5208	 Percent... 95.02% 	2020-03-07 15:55:01.144137
progress... 93900	 error... 5208	 Percent... 95.13% 	2020-03-07 15:56:21.098404
progress... 94000	 error... 5211	 Percent... 95.23% 	2020-03-07 15:57:35.471597
progress... 94100	 error... 5213	 Percent... 95.33% 	2020-03-07 15:58:52.586483
progress... 94200	 error... 5215	 Percent... 95.43% 	2020-03-07 16:00:05.061864
progress... 94300	 error... 5220	 Percent... 95.53% 	2020-03-07 16:01:20.477484
progress... 94400	 error... 5224	 Percent... 95.63% 	2020-03-07 16:02:35.231682
progress... 94500	 error... 5229	 Percen

In [24]:
data_f.to_csv("Recipe.csv", encoding = 'utf-8-sig', index = False)

In [134]:
sql_test = """
    create table test(
    id int unsigned not null auto_increment,
    name varchar(100) not null,
    primary key(id)
    );
"""

In [138]:
num = 12
sql = "insert into test (id, name) values(%s, %s)"
cursor.execute(sql, (num, meterial_meterial(2)))

InternalError: (1241, 'Operand should contain 1 column(s)')

In [102]:
# 1. 음식 제목 (감자탕)
# 2. 정보 (음식양, 조리시간, 조리 난이도)
# 3. 재료 (재료, 양념)
# 4. 요리 후기 (100)
# 5. 별점 평균 (4.9)
# 6. 조회수(10,000)
# 7. 조리 순서(7(단계))

# SQL 테이블 만들기
sql_create = """
    CREATE TABLE recipe(
        id INT UNSIGNED NOT NULL AUTO_INCREMENT,
        name VARCHAR(20) NOT NULL,
        information VARCHAR(30) NOT NULL,
        ingredient VARCHAR(100) NOT NULL,
        review_count NUMERIC(6) NOT NULL,
        score_mean FLOAT(3) NOT NULL,
        views NUMERIC(8) NOT NULL,
        cook_num NUMERIC(2) NOT NULL,
        PRIMARY KEY(id)
        );
"""

# SQL 연결

In [135]:
db = pymysql.connect(host = 'localhost', user = 'root', password = 'root', db = 'mysql', charset = 'utf8')
cursor = db.cursor()

In [136]:
cursor.execute("use world")

0

In [137]:
cursor.execute(sql_test)

0

In [118]:
cursor.execute('SELECT * FROM recipe')

0

In [89]:
cursor.execute("show tables")

5

In [123]:
titles()
information()
meterial_meterial(2)
score_count()
score_mean()
view_num()
cook_num()
num = 1
sql = "insert into recipe (id, name, information, ingredient, review_count, score_mean, views, cook_num) values(%s, %s, %s, %s, %s, %s, %s, %s)"
cursor.execute(sql, (num, titles(), information(), meterial_meterial(2), score_count(), score_mean(), view_num(), cook_num()))

InternalError: (1241, 'Operand should contain 1 column(s)')

In [148]:
db.commit()

In [150]:
db.close()

Error: Already closed

In [119]:
result = cursor.fetchall()
result

()

In [16]:
type(result)

tuple

In [14]:
for data in result:
    print(data[0])
    print(data[1])
    print(data[2])
    print(data[3])

1
i5
7700
Kaby Lake
